In [3]:
import math
import cv2
import numpy as np

In [4]:
def CalGaussianFunc(x,y,Sigma): #Calculate Gaussian Function
    G = 1/(2*math.pi*Sigma*Sigma)*math.exp(-(x*x+y*y)/(2*Sigma*Sigma))
    return G

In [5]:
def GaussianKer(Sigma): #Creating the Gaussian Kernel
    Gk = list()
    Gker = list()
    NorGK = list()
    max = 0
    for i in range(-3,4):
        for j in range(-3,4):
            C = CalGaussianFunc(i,j,Sigma)
            Gk.append(C)        
    for i in Gk:
        if i>max:
            max = abs(i)
    for i in Gk:        
        NorGK.append(abs(i)/max)
    Gker = [NorGK[i:i+7] for i in range(0, len(Gk), 7)] #Reshape to get gaussian kernel
    return Gker

In [6]:
def Pad(img):
    Ix=list()
    for line in img:
        line = list(line)
        line.insert(0,0)   #Column Padding to perform convolution
        line.insert(0,0)
        line.insert(0,0)
        line.insert(len(line),0)
        line.insert(len(line),0)
        line.insert(len(line),0)
        Ix.append(line)
    Opx = list()
    RowPad = [ 0 for i in range(len(Ix[0])) ] #Row padding to perform convolution
    Opx.append(RowPad)
    Opx.append(RowPad)
    Opx.append(RowPad)
    for i in Ix:
        Opx.append(i)
    Opx.append(RowPad) 
    Opx.append(RowPad) 
    Opx.append(RowPad)
    Opx = np.asarray(Opx)
    return Opx

In [7]:
def Convo(Gker,img):  #Performing Convolution
    
    Image = [[0]*img.shape[1] for i in range(img.shape[0])]
    img = Pad(img)
    Image = np.asarray(Image)
    for i in range(3,img.shape[0]-3):     # Loop over every pixel of the image
        for j in range(3,img.shape[1]-3):
            x = img[i-3:i+4,j-3:j+4]
            result = [[a*b for a, b in zip(k, l)] for k, l in zip(x, Gker)] #Element wise multiplication between the kernel and the sliced part of the image(For that pixel)
            elex = 0
            for row in result:
                for ele in row: 
                    elex = elex+ele #Adding the all the products to generate the pixel value for that pixel
    
            Image[i-3][j-3] = elex
    Opx_norm = [[0.0]*Image.shape[1] for i in range(Image.shape[0])] 
    Opx_norm = np.asarray(Opx_norm)
    maxval = 0
    for i in Image:      #Normalize after convolution
        for j in i:
            if maxval<abs(j):
                 maxval = j
    for i in range(0,Image.shape[0]):
        for j in range(0,Image.shape[1]):
            Opx_norm[i,j] = abs(Image[i,j])*255/maxval
    return Opx_norm   

In [17]:
def ResizeImage(img): #Resizing image for subsequent octaves
    Image = [[0]*int((img.shape[1]/2)) for i in range(int((img.shape[0]/2)))]
    Image = np.asarray(Image)
    k = l = 0
    for i in range(0,img.shape[0]-1,2):
        l=0
        for j in range(0,img.shape[1]-1,2):
                Image[k,l] = img[i,j]
                l=l+1
        k=k+1
        
    return Image    

In [9]:
def DoG(img1,img2):  #Function to calculate the Difference of Gaussians 
    Image = [[0.0]*img1.shape[1] for i in range(img1.shape[0])]
    Image = np.asarray(Image)
    Sum = 0
    #Image = img1 - img2
    for i in range(0,img1.shape[0]):
        for j in range(0,img1.shape[1]):
            Image[i,j] = img1[i,j] - img2[i,j]  #Element wise subtraction
       #     Sum = Sum + Image[i,j]
    Max = np.max(Image)
    for k in range(0,img1.shape[0]):                 #Normalized DoG to display it
        for l in range(0,img1.shape[1]):
            Image[k,l] = Image[k,l]*255/Max
            
    return Image

In [10]:
def FindKeyPoint(DoG1,DoG2,DoG3):  #Function to find ketpoints
    X = list()
    KeyPoint = list()
    for i in range(1,DoG1.shape[0]-1):
        for j in range(1,DoG1.shape[1]-1):
            x1 = DoG1[i-1:i+2,j-1:j+2]
            x2 = DoG2[i-1:i+2,j-1:j+2]
            x3 = DoG3[i-1:i+2,j-1:j+2]
            KeyPointx = i
            KeyPointy = j
            X = []
            for k in range(3):
                for l in range(3):
                        X.append(x1[k,l])
                        X.append(x2[k,l])
                        X.append(x3[k,l])
            Min = min(X)        
            Max = max(X)
            if x2[1,1]==Min or x2[1,1]==Max:
                KeyPoint.append([KeyPointx,KeyPointy])
    return KeyPoint            

In [45]:
def ShowKeyPoint(KeyPoint,scale,Img): #Function to show keypoints on the original image
    for x in range(len(KeyPoint)):
        pt = KeyPoint[x]
        Img[pt[0]*scale,pt[1]*scale] = 255
        
    return Img

In [48]:
def FindLeftmostKey(Keypoint):
    Left = list()
    for i in range(5):
        Left.append(Keypoint[i])
        
    return Left    

In [49]:
Img = cv2.imread("task2.jpg",0)   #Read image
Sigma1 = [1/math.sqrt(2),1,math.sqrt(2),2,2*math.sqrt(2)] #1st Scale
Sigma2 = [math.sqrt(2),2,2*math.sqrt(2),4,4*math.sqrt(2)] #2nd Scale
Sigma3 = [2*math.sqrt(2),4,4*math.sqrt(2),8,8*math.sqrt(2)] #3rd Scale
Sigma4 = [4*math.sqrt(2),8,8*math.sqrt(2),16,16*math.sqrt(2)] #4th Scale
Filename1 = ["octave4.1.png","octave4.2.png","octave4.3.png","octave4.4.png","octave4.5.png"]
Filename2 = ["octave3.1.png","octave3.2.png","octave3.3.png","octave3.4.png","octave3.5.png"]

N = Img.shape[0]
M = Img.shape[1]
Oct1 = list()
Oct2 = list()
Oct3 = list()
Oct4 = list()

#1st Octave
for i in range(5):
    K= GaussianKer(Sigma1[i])    
    Op = Convo(K,Img) 
    Oct1.append(Op)
    
#2nd Octave    
Image_by_two = ResizeImage(Img)
for i in range(5):
    K= GaussianKer(Sigma2[i])    
    Op = Convo(K,Image_by_two)
    Oct2.append(Op)
    

#3rd Octave
Image_by_four = ResizeImage(Image_by_two)
for i in range(5):
    K= GaussianKer(Sigma3[i])    
    Op = Convo(K,Image_by_four) 
    cv2.imwrite(Filename2[i],Op)
    Oct3.append(Op)

#4th Octave
Image_by_eight = ResizeImage(Image_by_four)
for i in range(5):
    K= GaussianKer(Sigma4[i])    
    Op = Convo(K,Image_by_eight)
    cv2.imwrite(Filename1[i],Op)
    Oct4.append(Op)

In [19]:
#1st DoG
DoG11 = DoG(Oct1[0],Oct1[1])   
DoG12 = DoG(Oct1[1],Oct1[2])
DoG13 = DoG(Oct1[2],Oct1[3])
DoG14 = DoG(Oct1[3],Oct1[4])
#2nd DoG 
DoG21 = DoG(Oct2[0],Oct2[1])
DoG22 = DoG(Oct2[1],Oct2[2])
DoG23 = DoG(Oct2[2],Oct2[3])
DoG24 = DoG(Oct2[3],Oct2[4])
#3rd DoG
DoG31 = DoG(Oct3[0],Oct3[1])
DoG32 = DoG(Oct3[1],Oct3[2])
DoG33 = DoG(Oct3[2],Oct3[3])
DoG34 = DoG(Oct3[3],Oct3[4])
#4th DoG
DoG41 = DoG(Oct4[0],Oct4[1])
DoG42 = DoG(Oct4[1],Oct4[2])
DoG43 = DoG(Oct4[2],Oct4[3])
DoG44 = DoG(Oct4[3],Oct4[4])

In [20]:
#1st Keypoints
KeyPoint1 = FindKeyPoint(DoG11,DoG12,DoG13)
KeyPoint2 = FindKeyPoint(DoG12,DoG13,DoG14)
#2nd Keypoints
KeyPoint3 = FindKeyPoint(DoG21,DoG22,DoG23)
KeyPoint4 = FindKeyPoint(DoG22,DoG23,DoG24)
#3rd Keypoints
KeyPoint5 = FindKeyPoint(DoG31,DoG32,DoG33)
KeyPoint6 = FindKeyPoint(DoG32,DoG33,DoG34)
#4th Keypoints
KeyPoint7 = FindKeyPoint(DoG41,DoG42,DoG43)
KeyPoint8 = FindKeyPoint(DoG42,DoG43,DoG44)

In [46]:
Img = cv2.imread("task2.jpg",0)  
#1st Set of KeyPoints
Img11 = ShowKeyPoint(KeyPoint1,1,Img) 
Img12 = ShowKeyPoint(KeyPoint2,1,Img)
#2nd Set of KeyPoints
Img21 = ShowKeyPoint(KeyPoint3,2,Img)
Img22 = ShowKeyPoint(KeyPoint4,2,Img)
#3rd Set of KeyPoints
Img31 = ShowKeyPoint(KeyPoint5,4,Img)
Img32 = ShowKeyPoint(KeyPoint6,4,Img)
#4th Set of KeyPoints
Img41 = ShowKeyPoint(KeyPoint7,8,Img)
Img42 = ShowKeyPoint(KeyPoint8,8,Img)

In [50]:
cv2.imwrite("Keypoints11.png",Img11) #Show keypoints for the 1st octave
cv2.imwrite("Keypoints12.png",Img12)
cv2.imwrite("Keypoints21.png",Img21)
cv2.imwrite("Keypoints22.png",Img22)
cv2.imwrite("Keypoints31.png",Img31)
cv2.imwrite("Keypoints32.png",Img32)
cv2.imwrite("Keypoints41.png",Img41)
cv2.imwrite("Keypoints42.png",Img42)

True

In [51]:
L = FindLeftmostKey(KeyPoint1)
print(L)

[[1, 1], [1, 35], [1, 79], [1, 112], [1, 136]]


In [62]:
cv2.imwrite("DoG43.png",DoG43)

True

In [61]:
cv2.imwrite("Keypoints31.png",Img31)

True